In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [2]:
transactions = pd.read_json(r'transactions.txt',lines=True)
transactions

,customer,date,itemList,store
0,10,2017-04-06 12:09:32,"[{'item': '20126907_EA', 'price': 1.88, 'quant...",825f9cd5f0390bc77c1fed3c94885c87
1,100,2017-01-10 12:59:09,"[{'item': '20132638_KG', 'price': 3.33, 'quant...",a666587afda6e89aec274a3657558a27
2,1000,2017-04-17 18:53:40,"[{'item': '20788909_EA', 'price': 3.49, 'quant...",ebb71045453f38676c40deb9864f811d
3,100034,2017-01-29 12:32:04,"[{'item': '20986479_EA', 'price': 9.98, 'quant...",3dd48ab31d016ffcbf3314df2b3cb9ce
4,100038,2017-04-11 18:26:28,"[{'item': '20962514_EA', 'price': 4.85, 'quant...",b132ecc1609bfcf302615847c1caa69a
...,...,...,...,...
1377438,99940,2017-03-02 13:40:38,"[{'item': '20801173_EA', 'price': 4.38, 'quant...",321cf86b4c9f5ddd04881a44067c2a5a
1377439,99953,2017-05-01 20:37:40,"[{'item': '20432217001_EA', 'price': 4.87, 'qu...",82965d4ed8150294d4330ace00821d77
1377440,99959,2017-02-03 14:44:19,"[{'item': '20821361001_EA', 'price': 2.77, 'qu...",c0d8ec4888d56b0fabfe476c780e2cc4
1377441,9997,2017-06-08 19:34:11,"[{'item': '20016320_EA', 'price': 2.99, 'quant...",7fe1f8abaad094e0b5cb1b01d712f708


In [3]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377443 entries, 0 to 1377442
Data columns (total 4 columns):
customer    1377443 non-null int64
date        1377443 non-null datetime64[ns]
itemList    1377443 non-null object
store       1377443 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 42.0+ MB


In [4]:
# Monthly transcation count

print (transactions.date.min())
print (transactions.date.max())

month_tansactions = transactions.groupby([pd.Grouper(key='date', freq='M')])['date'].transform('size').astype(int)
set(month_tansactions)


2017-01-01 08:54:27
2017-06-30 23:57:35


{185490, 221924, 229292, 232744, 251654, 256339}

In [5]:
# Get all the item list from the transaction data
item_list = transactions['itemList'].to_frame()
item_list 

,itemList
0,"[{'item': '20126907_EA', 'price': 1.88, 'quant..."
1,"[{'item': '20132638_KG', 'price': 3.33, 'quant..."
2,"[{'item': '20788909_EA', 'price': 3.49, 'quant..."
3,"[{'item': '20986479_EA', 'price': 9.98, 'quant..."
4,"[{'item': '20962514_EA', 'price': 4.85, 'quant..."
...,...
1377438,"[{'item': '20801173_EA', 'price': 4.38, 'quant..."
1377439,"[{'item': '20432217001_EA', 'price': 4.87, 'qu..."
1377440,"[{'item': '20821361001_EA', 'price': 2.77, 'qu..."
1377441,"[{'item': '20016320_EA', 'price': 2.99, 'quant..."


In [6]:
#Data Exploration - No of unique items and stores
        
item_ids = [row[i]['item'] for row in item_list['itemList'] for i in range(len(row))] 
item_ids = list(set(item_ids))
print('Total unque IDs',len(item_ids))


stores = list(set(transactions['store']))
print('Total Stores',len(stores))



Total unque IDs 70771
Total Stores 1019


In [7]:
# Product data
products = pd.read_csv('products.txt', sep="\t", header=None)
products.columns = ["ID", "MCH", "name"]

products

,ID,MCH,name
0,20000002_EA,M10210701,Tuna Chunks in Broth
1,20000005_EA,M02270201,Fresh-Pressed Sweet Apple Cider
2,20000053_EA,M10210901,French Dijon Mustard
3,20000056001_KG,M02270304,Anaheim Peppers
4,20000068_KG,M05350101,Swiss Cheese
...,...,...,...
70766,21044783_KG,M03310801,"Turkey Breast, Homestyle Seasoning"
70767,21044784_KG,M03310801,Ground Turkey Homestyle Seasoning
70768,21045023_EA,M10020101,Fishin' 2+ Years
70769,21045741_EA,M03310402,Mild Italian Sausage


In [8]:
products.describe()

,ID,MCH,name
count,70771,70771,70771
unique,70771,379,60328
top,20133680_EA,M04330104,2% Milk
freq,1,1436,77


In [9]:

transaction_row = [[row[i]['item'] for i in range(len(row))]  for row in item_list['itemList']] 
transaction_row[:3]

[['20126907_EA',
  '20185742_EA',
  '20138681_EA',
  '20049778001_EA',
  '20419715007_EA',
  '20321434_EA',
  '20068076_KG',
  '20022893002_EA',
  '20299328003_EA'],
 ['20132638_KG',
  '20320042001_EA',
  '20320832003_EA',
  '20128148_KG',
  '20027478_KG',
  '20653232_EA',
  '20317755_EA',
  '20519704_KG',
  '20591843_KG'],
 ['20788909_EA', '20975073_EA', '20868904_EA', '20189092_EA']]

In [10]:
# Make a dataframe of all the purchased ids

item_ids = [row[i]['item'] for row in item_list['itemList'] for i in range(len(row))] 
item_ids_df = pd.DataFrame(item_ids,columns=['id']) 
item_ids_df


,id
0,20126907_EA
1,20185742_EA
2,20138681_EA
3,20049778001_EA
4,20419715007_EA
...,...
12084404,20316388_EA
12084405,20704361002_EA
12084406,20174632_EA
12084407,21004274_EA


In [11]:
# Select the ids which who are bought more than 300 times in six months
# This step is required to reduce the data size becaus of limited memory

v = item_ids_df.id.value_counts()
item_ids_df = item_ids_df[item_ids_df.id.isin(v.index[v.gt(300)])]
freq_ids = set(item_ids_df.id)


In [12]:
# check if the transaction row is subset of frequent list
final_transactions = []
for row in transaction_row:
    if(set(row).issubset(freq_ids)): 
        final_transactions.append(row)


In [13]:
# Create the one hot encoded sparse dataframe
te = TransactionEncoder()

oht_ary = te.fit(final_transactions).transform(final_transactions, sparse=True)
sparse_df = pd.SparseDataFrame(oht_ary, columns=te.columns_, default_fill_value=False)
sparse_df = sparse_df.astype("int")

sparse_df

c:\users\chandini\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """
c:\users\chandini\appdata\local\programs\python\python36\lib\site-packages\pandas\core\sparse\frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
c:\users\chandini\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3456: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>

,20000005_EA,20000093_EA,20000177_EA,20000207001_EA,20000356_EA,20000368_EA,20000433_EA,20000476001_EA,20000527_EA,20000533_EA,...,21025394_C12,21025519_EA,21025851_EA,21025975_EA,21026048_EA,21026058_EA,21026077_EA,21027142001_EA,21027889_EA,21035960001_EA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
430620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
430621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
430622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(sparse_df)

In [15]:
data_matrix

,20000005_EA,20000093_EA,20000177_EA,20000207001_EA,20000356_EA,20000368_EA,20000433_EA,20000476001_EA,20000527_EA,20000533_EA,...,21025394_C12,21025519_EA,21025851_EA,21025975_EA,21026048_EA,21026058_EA,21026077_EA,21027142001_EA,21027889_EA,21035960001_EA
20000005_EA,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.007039,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
20000093_EA,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
20000177_EA,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.006635,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.008787,0.000000,0.000000,0.00000,0.000000,0.000000
20000207001_EA,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
20000356_EA,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21026058_EA,0.0,0.0,0.0,0.0,0.0,0.0,0.003778,0.002996,0.004894,0.0,...,0.007142,0.000000,0.0,0.0,0.226153,1.000000,0.074227,0.00332,0.000000,0.007210
21026077_EA,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.016038,0.000000,0.0,0.0,0.124722,0.074227,1.000000,0.00000,0.000000,0.000000
21027142001_EA,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.003610,0.000000,0.0,...,0.000000,0.006558,0.0,0.0,0.000000,0.003320,0.000000,1.00000,0.000000,0.000000
21027889_EA,0.0,0.0,0.0,0.0,0.0,0.0,0.014185,0.000000,0.004594,0.0,...,0.000000,0.010217,0.0,0.0,0.000000,0.000000,0.000000,0.00000,1.000000,0.013535


In [16]:

def related_products(item_id):
    recommended_products = (data_matrix.loc[item_id].nlargest(7))
    skip = 1
    print('Item ID','\t\t','Related Products to:',list(products.loc[products['ID'] == item_id,'name']))
    for i in recommended_products.index:
        if skip == 0:
            print(i,'\t\t',list(products.loc[products['ID'] == i,'name']))
        skip = 0



In [17]:
related_products('20592676_EA')

Item ID 		 Related Products to: ['Celebration Cupcakes, Chocolate']
20379763_EA 		 ['Celebration Cupcakes, White']
20333714001_EA 		 ['Organic 100% Apple Juice']
20189092_EA 		 ['Plastic Bags']
20628489001_EA 		 ['Salad Dressing, Caesar']
20319988_EA 		 ['Sweet & Salty Chew Nut Bars, 6 Almond/6 Peanut/6 Cashew']
20336722002_EA 		 ['Pure Apple Juice']


In [18]:
related_products('20801754003_C15')

Item ID 		 Related Products to: ['7 Up']
20801754001_C15 		 ['Pepsi']
20801754002_C15 		 ['Diet Pepsi']
20561883003_C15 		 ['Sprite']
20348537002_EA 		 ['Disinfectant Wipes, Lemon']
20307938001_C12 		 ['Orange']
20561883001_C15 		 ['Coca Cola Classic, 15 Pack']
